In [ ]:
# Run in new virtual environment:
# pip install notebook
# python -m ipykernel install

In [ ]:
# Run in notebook
# %pip install python-dotenv requests

In [3]:
# Import libraries
import os
from dotenv import load_dotenv
import requests
import json
import sqlite3

In [4]:
class Portfolio:
    def __init__(self):
        self.cash_balance = 0
        self.positions = {}
    
    def add_cash(self, cash_amount):
        self.cash_balance += cash_amount
    
    def remove_cash(self, cash_amount):
        self.cash_balance -= cash_amount
    
    def add_position(self, symbol, quantity, purchase_price):
        if symbol in self.positions:
            self.positions[symbol]["quantity"] += quantity
            self.positions[symbol]["average_price"] = 0 # implement logic
        else:
            self.positions[symbol] = {
                "quantity": quantity,
                "average_price": purchase_price
            }
        
        self.cash_balance -= quantity * purchase_price
    
    def remove_position(self, symbol, quantity):
        if symbol not in self.positions:
            raise ValueError(f"Position {symbol} not found in portfolio.")

        current_quantity = self.positions[symbol]["quantity"]
        if quantity > current_quantity:
            raise ValueError(f"Cannot remove {quantity} shares of {symbol}. Only {current_quantity} shares are held.")

        if quantity == current_quantity:
            del self.positions[symbol]
        else:
            self.positions[symbol]["quantity"] -= quantity
            # Here, you'll likely need to adjust the 'purchase_data' 
            # to reflect the shares being sold. The specific logic will 
            # depend on your accounting method (e.g., FIFO, LIFO).

        # Update cash balance (assuming a sell transaction at current market price)
        current_price = self.get_current_price(symbol)  # You'll need to implement this method
        self.cash_balance += quantity * current_price

    def get_total_value(self):
        total_value = self.cash_balance
        for symbol in self.positions:
            total_value += symbol.quantity * symbol.purchase_data.priceposition()
        return total_value

In [21]:
class DataHandler:
    def __init__(self, data_source = 'Alpha Vantage'):
        self.data_source = data_source
        load_dotenv()
        self.api_key = os.environ.get('ALPHA_VANTAGE_API_KEY')
    
    def get_last_close(self, symbol):
        if self.data_source == 'Alpha Vantage':
            base_url = 'https://www.alphavantage.co/query?'
            params = {
                'function': 'GLOBAL_QUOTE',
                'symbol': symbol,
                'apikey': self.api_key
            }
            response = requests.get(base_url, params=params)
            price = response.json()["Global Quote"]["05. price"]
            return price
    
    def get_historical(self, symbol, start_date, end_date):
        if self.data_source == 'Alpha Vantage':
            base_url = 'https://www.alphavantage.co/query?'
            params = {
                'function': 'TIME_SERIES_DAILY',
                'symbol': symbol,
                'apikey': self.api_key
            }
            response = requests.get(base_url, params=params)
            return response.json()

In [26]:
# class PerformanceAnalyser:
#     def __init__(self, portfolio):
#         self.portfolio = portfolio
    
#     def calculate_cumulative_returns(self):
#         cumulative_returns = (1 + self.returns).cumprod() - 1
#         return cumulative_returns
    
#     def calculate_annualized_returns(self):
#         total_returns = self.returns.sum()
#         num_periods = len(self.returns)
#         annualized_returns = (1 + total_returns) ** (252 / num_periods) - 1
#         return annualized_returns
    
#     def calculate_volatility(self):
#         volatility = self.returns.std() * (252 ** 0.5)
#         return volatility
    
#     def calculate_sharpe_ratio(self, risk_free_rate):
#         excess_returns = self.returns - risk_free_rate
#         sharpe_ratio = excess_returns.mean() / excess_returns.std() * (252 ** 0.5)
#         return sharpe_ratio

In [27]:
# class RiskAnalyser:
#     def __init__(self, portfolio):
#         self.portfolio = portfolio

#     def calculate_volatility(self):
#         volatility = np.std(self.returns) * np.sqrt(252)
#         return volatility

#     def calculate_beta(self, benchmark_returns):
#         covariance = np.cov(self.returns, benchmark_returns)[0][1]
#         benchmark_variance = np.var(benchmark_returns)
#         beta = covariance / benchmark_variance
#         return beta

In [28]:
# class PortfolioOptimiser:
#   def __init__(self, portfolio, risk_reward):
#     self.portfolio = portfolio
#     self.risk_reward = risk_reward

#   def optimise_weights(self):
#     # Perform portfolio optimisation here
#     # Access portfolio attributes and risk model for optimisation
#     # Example placeholder code
#     optimal_weights = [0.25, 0.25, 0.25, 0.25]  # Placeholder example weights
#     return optimal_weights

In [22]:
portfolio = Portfolio()

assets = {
    "GOOGL": {
        "shares": 30,
        "purchase_price": 159.53
    },
    "MSFT": {
        "shares": 10,
        "purchase_price": 404.60
    },
    "BRK-B": {
        "shares": 9,
        "purchase_price": 445.98
    },
    "ADBE": {
        "shares": 3,
        "purchase_price": 516.66
    }
}

for symbol, data in assets.items():
    portfolio.add_position(symbol, data["shares"], data["purchase_price"])
    
print(portfolio.positions)

{'GOOGL': {'quantity': 30, 'average_price': 159.53}, 'MSFT': {'quantity': 10, 'average_price': 404.6}, 'BRK-B': {'quantity': 9, 'average_price': 445.98}, 'ADBE': {'quantity': 3, 'average_price': 516.66}}


In [23]:
data_handler = DataHandler()

for symbol in portfolio.positions:
    last_close = data_handler.get_last_close(symbol)
    print(symbol, ":", last_close)

GOOGL : 163.9500
MSFT : 428.0200
BRK-B : 457.4700
ADBE : 515.4800


In [ ]:
# conn = sqlite3.connect('stock_data.db')
# cursor = conn.cursor()